# Diccionario Quechua

Análisis de diccionarios del idioma Quechua, con fuentes en varios formatos, gráficos informativos y conclusiones.

## Índice

1. Definición del problema
1. Plantear los objetivos
1. Obtener los datos
1. Preparar y optimizar los datos
1. Procesar y analizar los datos
1. Conclusiones

## Desarrollo

### Definición del problema

El idioma quechua tiene dos vertientes en el uso de tres o cinco vocales, 
así como el número de sufijos usados.

### Plantear los objetivos

1. Describir las palabras en idioma Quechua según vocales y sufijos.
2. Listar las palabras en el idioma Quechua.
3. Clasificar según las tres vocales diferenciando los sufijos.

### Obtener los datos

El idioma Quechua cuenta con presencia en Internet y se dispone de diccionarios en varios formatos:

* Publicaciones de instituciones o autores individuales en formatos PDF, ePUB, etc.
* En formato para su fácil uso en Internet como Txt, Zip, etc.

In [49]:
# Descargar los archivos de datos
# !pip install requests

import os
import requests

data = {
    'runasimi.txt': 'https://www.runasimi.de/runasimi.txt',
    'runasimi.zip': 'https://www.runasimi.de/runasimi.zip',
    'diccionario.pdf': 'https://fcctp.usmp.edu.pe/librosfcctp/DICCIONARIO-Quechua-espanol-VOL_2.pdf'
}

if not os.path.exists('./data'):
    os.makedirs('./data')

for fname, url  in data.items():
    fpath = './data/' + fname
    print(fpath)
    if not os.path.exists(fpath):
        with requests.get(url, stream=True, verify=False) as r:
            r.raise_for_status()
            with open(fpath, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)

./data/runasimi.txt
./data/runasimi.zip
./data/diccionario.pdf


### Preparar y optimizar los datos

In [50]:
# Leer documento en texto separado por tabs (TSV)
# !pip install pandas

import pandas as pd

f_txt = './data/runasimi.txt'

df_txt = pd.read_csv(f_txt, sep='\t', encoding='latin1')
print(df_txt.columns)

Index(['Hanan Runasimi', 'Ima simi', 'Sikllalla Runasimi', 'QUS-aiux',
       'BuL-aiux', 'Tupah Yupanki (Qusqu)', 'Tukuman', 'Ayakuchu',
       'Qhochapampa', 'Qullasuyu', 'QUS-BuL-aiu', 'Qusqu', 'Qosqo',
       'Cajamarca (Coombs)', 'Qasamarka', 'Inkawasi', 'Laraos', 'Lamista',
       'Imbabura (runashimi killkay)', 'Quichua (esp. quillcai)', 'Wanka',
       'Jauja Wanka', 'Huánuco (Weber)', 'Wanuku', 'Ankash (ay, aw)',
       'Ankash (ee, oo)', 'Kurunku', 'Deutsch', 'English', 'Español',
       'Italiano', 'Français', 'Dansk/Norsk', 'Aymara',
       'Yurap uywap salqap amutaypaq sutin', 'Kaqlla simi'],
      dtype='object')


In [51]:
# Un archivo Excel dentro de un Zip comprimido
# !pip install xlrd

import pandas as pd
import zipfile


f_zip = './data/runasimi.zip'
f_xls = 'runasimi.xls'

with zipfile.ZipFile(f_zip, 'r') as z:
    with z.open(f_xls) as f:
        df_xls = pd.read_excel(f)
        print(df_xls.columns)

Index(['Hanan Runasimi', 'Ima simi', 'Sikllalla Runasimi', 'QUS-aiux',
       'BuL-aiux', 'Tupah Yupanki (Qusqu)', 'Tukuman', 'Ayakuchu',
       'Qhochapampa', 'Qullasuyu', 'QUS-BuL-aiu', 'Qusqu', 'Qosqo',
       'Cajamarca (Coombs)', 'Qasamarka', 'Inkawasi', 'Laraos', 'Lamista',
       'Imbabura (runashimi killkay)', 'Quichua (esp. quillcai)', 'Wanka',
       'Jauja Wanka', 'Huánuco (Weber)', 'Wanuku', 'Ankash (ay, aw)',
       'Ankash (ee, oo)', 'Kurunku', 'Deutsch', 'English', 'Español',
       'Italiano', 'Français', 'Dansk/Norsk', 'Aymara', 'Avañee', 'Русский',
       'Yurap uywap salqap amutaypaq sutin', 'Kaqlla simi', 'Sut'inchaku',
       'Ima mallki', 'Explicación del uso de la planta',
       'Gonçalez Holguin y Huaman Poma', 'Quechua clásico', 'Waruchiri',
       'Guaro Cheri', 'Tarma', 'Pastasa', 'Napuruna', 'Runasimi C',
       'Runasimi Nord'],
      dtype='object')


In [52]:
# Leer contenido en un PDF
# !pip install pdfplumber

import pdfplumber

f_pdf = './data/diccionario.pdf'

with pdfplumber.open(f_pdf) as pdf:
    print('Pages', len(pdf.pages))
    print('Page 2 =>', pdf.pages[2].extract_text())

Pages 1443
Page 2 => Julio Calvo Pérez es catedrático jubilado de Lingüística General en la Universitat de
València (España). Ha publicado diversas obras sobre castellano peninsular y peruano
en sus ramas académica y andina, especialmente de corte lexicográfico. Su andadura en
torno al quechua comenzó al principio de los años noventa y fraguó en su Pragmática
y gramática del quechua cuzqueño (1993). Sus más de cien publicaciones sobre la
lengua andina subrayan su dedicación a diversos temas de gramática, pragmática,
morfología y sintaxis, fonología y lexicografía, como ahora la segunda edición de su
monumental obra Nuevo diccionario español-quechua, quechua-español, publicada en
cinco volúmenes por primera vez en 2009. Ha publicado una traducción al español
de Ollantay (1998) y otra al catalán (2004), en colaboración con el novelista Salvador
Company. También ha traducido al castellano la pieza teatral Uska Pawqar Inka (2019).
Es académico correspondiente de la Academia Peruana de la L


### Procesar y analizar los datos

### Conclusiones